* U.S. Manufactured Goods: Domestic Production vs Imports (2013–2024)

In [18]:
%pip install --upgrade altair

  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
Using cached altair-5.5.0-py3-none-any.whl (731 kB)
  Attempting uninstall: altair
    Found existing installation: altair 5.0.1
    Uninstalling altair-5.0.1:
      Successfully uninstalled altair-5.0.1
Note: you may need to restart the kernel to use updated packages.


In [60]:
# policy dataframe
import pandas as pd

policies = pd.DataFrame([
    {'start':'2015-01-01', 'end':'2016-06-30', 'policy':'A'},
    {'start':'2016-07-01', 'end':'2018-03-31', 'policy':'B'},
    {'start':'2018-04-01', 'end':'2020-12-31', 'policy':'C'},
])
policies['start'] = pd.to_datetime(policies['start'])
policies['end']   = pd.to_datetime(policies['end'])

In [61]:
import pandas as pd
import altair as alt

# prepare dataframe
df_imp = pd.read_csv('../data/Processed/import_domestic/import.csv', names=['Quarter','Import_Value'],skiprows=1)
df_dmst  = pd.read_csv('../data/Processed/import_domestic/domestic_gross_output.csv', names=['Quarter','Domestic_Gross_Output_Value'],skiprows=1)

df = df_imp.merge(df_dmst, on='Quarter')
df['Date'] = pd.PeriodIndex(df['Quarter'], freq='Q').to_timestamp()
df['Share']   = df['Import_Value'] / (df['Import_Value'] + df['Domestic_Gross_Output_Value']) * 100

df_bar = df.melt(
    id_vars=['Date'],
    value_vars=['Domestic_Gross_Output_Value','Import_Value'],
    var_name='Type',
    value_name='Amount'
)

# build selector
nearest = alt.selection_point(
    fields=['Date'],
    nearest=True,
    on='mouseover',
    empty='none'
)

# build policy row
policy_bars = alt.Chart(policies).mark_rect().encode(
    x='start:T',
    x2='end:T',
    y=alt.value(0),        
    y2=alt.value(1),
    color=alt.Color('policy:N',
                    scale=alt.Scale(scheme='category10'))
).properties(height=30)

policy_labels = alt.Chart(policies).mark_text(
    baseline='middle', dy=-10, color='black'
).encode(
    x=alt.X('start:T', title=None),   
    text='policy:N'
).properties(height=30)

policy_row = alt.layer(policy_bars, policy_labels).properties(
    width=800,   
    title='Policy'
)

# build top
bars = alt.Chart(df_bar).mark_bar().encode(
    x=alt.X('Date:T',),
    y=alt.Y('Amount:Q', title='Absolute Value (Billion $)'),
    color=alt.Color('Type:N',
                    scale=alt.Scale(domain=['Domestic_Gross_Output_Value','Import_Value'],
                                    range=['#4C78A8','#F58518']),
                    legend=alt.Legend(title='Category'))
)

bars_points = alt.Chart(df_bar).mark_point(size=100, opacity=0).encode(
    x='Date:T',
    y='Amount:Q'
).add_selection(nearest)

bars_rule = alt.Chart(df).mark_rule(color='gray').encode(
    x='Date:T'
).transform_filter(nearest)

bars_text = alt.Chart(df_bar).transform_filter(nearest).mark_text(
    dy=-5
).encode(
    x='Date:T',
    y='Amount:Q',
    text=alt.Text('Amount:Q', format='.0f'),
    detail='Type:N'      
)

top = alt.layer(bars, bars_points, bars_rule, bars_text).properties(
    width=800, height=300,
    title='Absolute Value: Import vs Domestic'
)

# build bottom
raw_line = alt.Chart(df).mark_line(
    color='steelblue', strokeWidth=2
).encode(
    x='Date:T',
    y=alt.Y('Share:Q', title='Import Share (%)'),
)

points = alt.Chart(df).mark_point(
    color='steelblue', size=50
).encode(
    x='Date:T',
    y='Share:Q'
)

trend = alt.Chart(df).transform_regression(
    'Date', 'Share', method='linear'
).mark_line(
    color='red', strokeDash=[4,2], strokeWidth=2
).encode(
    x='Date:T',
    y='Share:Q'
)

line_rule = alt.Chart(df).mark_rule(color='gray').encode(
    x='Date:T'
).transform_filter(nearest)

line_text = alt.Chart(df).transform_filter(nearest).mark_text(
    dy=-10, color='black'
).encode(
    x='Date:T',
    y='Share:Q',
    text=alt.Text('Share:Q', format='.2f')
)

bottom = alt.layer(raw_line, points, trend, line_rule, line_text).properties(
    width=800, height=200,
    title='Import Share'
)

# assemble
chart = alt.vconcat(policy_row, top, bottom).resolve_scale(
    x='shared',    
    y='independent' 
)

chart

/opt/homebrew/anaconda3/lib/python3.12/site-packages/altair/vegalite/v5/api.py:362: AltairDeprecationWarning:

The value of 'empty' should be True or False.

/opt/homebrew/anaconda3/lib/python3.12/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning:

'add_selection' is deprecated. Use 'add_params' instead.

/opt/homebrew/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning:

the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.

/opt/homebrew/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning:

the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.

/opt/homebrew/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning:

the convert_dtype parameter is deprecated and will be remov

alt.VConcatChart(...)